In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install langchain_community
!pip install pypdfium2
!pip install langchain_experimental
!pip install kiwipiepy
!pip install faiss-gpu-cu12
!pip install rank_bm25

In [13]:
import os
from langchain.docstore.document import Document

root_folder = "C:/Users/minkyu/Desktop/dacon accident prevention/진짜 시작/건설안전지침_md"

all_documents = []  # Document 객체 리스트

# 폴더를 순회하며 .md 파일 처리
for current_path, dirs, files in os.walk(root_folder):
    # 현재 경로가 루트 폴더와 다른 경우(=서브폴더), 폴더명을 메타데이터 title로 사용
    if current_path != root_folder:
        folder_title = os.path.basename(current_path)
    else:
        folder_title = ""

    # 현재 폴더의 .md 파일 목록
    md_files = [f for f in files if f.lower().endswith('.md')]

    # page 번호를 추출해 정렬하기 위한 헬퍼 함수
    def extract_page_number(filename):
        # 예: "page_1.md" -> "page_1" -> 뒤의 "1" -> int(1)
        base = os.path.splitext(filename)[0]  # "page_1"
        return int(base.split('_')[-1])       # "1"

    # .md 파일 정렬
    md_files = sorted(md_files, key=extract_page_number)

    # 첫 3개(page_1, page_2, page_3)는 무시하고, 그 이후 파일들만
    md_files_to_read = md_files[3:]

    # 페이지 단위로 Document 생성
    for i, md_file in enumerate(md_files_to_read, start=4):  # 시작 인덱스를 4로 설정
        md_file_path = os.path.join(current_path, md_file)
        with open(md_file_path, "r", encoding="utf-8") as f:
            page_content = f.read()

        # 메타데이터에 폴더명과 실제 페이지 번호(i)를 저장
        doc_metadata = {
            "title": folder_title if folder_title else "NoTitle",
            "page_num": i
        }

        doc = Document(page_content=page_content, metadata=doc_metadata)
        all_documents.append(doc)

        print(f"폴더: {folder_title}, 파일: {md_file}, 페이지 번호: {i}")

print(f"\n총 Document 수: {len(all_documents)}")


폴더: F.C.M 교량공사 안전보건작업 지침, 파일: page_4.md, 페이지 번호: 4
폴더: F.C.M 교량공사 안전보건작업 지침, 파일: page_5.md, 페이지 번호: 5
폴더: F.C.M 교량공사 안전보건작업 지침, 파일: page_6.md, 페이지 번호: 6
폴더: F.C.M 교량공사 안전보건작업 지침, 파일: page_7.md, 페이지 번호: 7
폴더: F.C.M 교량공사 안전보건작업 지침, 파일: page_8.md, 페이지 번호: 8
폴더: F.C.M 교량공사 안전보건작업 지침, 파일: page_9.md, 페이지 번호: 9
폴더: F.C.M 교량공사 안전보건작업 지침, 파일: page_10.md, 페이지 번호: 10
폴더: F.C.M 교량공사 안전보건작업 지침, 파일: page_11.md, 페이지 번호: 11
폴더: F.C.M 교량공사 안전보건작업 지침, 파일: page_12.md, 페이지 번호: 12
폴더: F.C.M 교량공사 안전보건작업 지침, 파일: page_13.md, 페이지 번호: 13
폴더: F.C.M 교량공사 안전보건작업 지침, 파일: page_14.md, 페이지 번호: 14
폴더: F.C.M 교량공사 안전보건작업 지침, 파일: page_15.md, 페이지 번호: 15
폴더: F.C.M 교량공사 안전보건작업 지침, 파일: page_16.md, 페이지 번호: 16
폴더: F.C.M 교량공사 안전보건작업 지침, 파일: page_17.md, 페이지 번호: 17
폴더: F.C.M 교량공사 안전보건작업 지침, 파일: page_18.md, 페이지 번호: 18
폴더: F.C.M 교량공사 안전보건작업 지침, 파일: page_19.md, 페이지 번호: 19
폴더: F.C.M 교량공사 안전보건작업 지침, 파일: page_20.md, 페이지 번호: 20
폴더: F.C.M 교량공사 안전보건작업 지침, 파일: page_21.md, 페이지 번호: 21
폴더: F.C.M 교량공사 안전보건작업 지침, 파일: page_22.md, 페이지 번호: 22
폴더: F

In [14]:
embeddings = HuggingFaceEmbeddings(model_name="jhgan/ko-sbert-sts")

content_index = FAISS.from_documents(all_documents, embeddings)
content_index.save_local("faiss_content_index")

print("FAISS content index saved successfully.")

FAISS content index saved successfully.


In [5]:
import pandas as pd
train_dataset = pd.read_csv('train_df.csv')

In [34]:
combined_training_data = train_dataset.apply(
    lambda row: {
        "text": (
            f"{row['인적사고']} 에 대한 안전조치사항 "
        ),
        "answer": row["재발방지대책 및 향후조치계획"]
    },
    axis=1
)

In [35]:
print(combined_training_data[0]['text'])


떨어짐(2미터 미만) 에 대한 안전조치사항 


In [28]:
# bm25 토크나이저로 한국어 토크나이저 사용 하기 위해서 불러옴
from kiwipiepy import Kiwi
kiwi = Kiwi()

def ko_kiwi_tokenizer(text: str):
    # Kiwi 토크나이저는 각 토큰에 대한 다양한 정보를 반환합니다.
    # 여기서는 토큰의 표면 형태(텍스트)만 추출합니다.
    tokens = kiwi.tokenize(text)
    return [token[0] for token in tokens]

In [30]:
from langchain.retrievers import BM25Retriever, EnsembleRetriever
from langchain.docstore.document import Document
# Retriever 정의
# pdf_chunks에서 Document 객체 리스트 생성
# pdf_chunks에서 Document 객체 리스트 생성
docs = all_documents
retriever = content_index.as_retriever(search_type="similarity", search_kwargs={"k": 3})
bm25_retriever = BM25Retriever.from_documents(docs, tokenizer=ko_kiwi_tokenizer , k =3 )

# 앙상블 리트리버 생성
ensemble_retriever = EnsembleRetriever(
    retrievers=[bm25_retriever, retriever],
    weights=[0.5, 0.5]  # 각 리트리버에 동일 가중치 부여 (가중치 합은 1.0)
)

In [31]:
combined_training_data[1]['text']

'로프를 이용 한 양중작업상태 불량으로 묶어 둔 배관 서포트가 지면으로 30Cm 미끄러지며 협착 발생. 에 대한 안전조치사항 '

In [32]:
import numpy as np

def cosine_similarity(vec1, vec2):
    return np.dot(vec1, vec2) / (np.linalg.norm(vec1) * np.linalg.norm(vec2))

In [38]:
n=9

# query와 정답 출력
query = combined_training_data[n]['text']
print('질문 :', query)
print('정답 :', combined_training_data[n]['answer'])

# query 임베딩 계산
query_embedding = embeddings.embed_query(query)

# 앙상블 리트리버로 검색 후 각 문서에 대해 코사인 유사도 계산 및 출력
results = ensemble_retriever.get_relevant_documents(query)

for doc in results:
    # 문서의 내용을 일부 출력
    
    # 문서 임베딩 계산 (문서 전체를 대상으로 임베딩을 생성합니다)
    doc_embedding = embeddings.embed_query(doc.page_content)
    similarity = cosine_similarity(query_embedding, doc_embedding)
    
    print(f"\n제목: {doc.metadata.get('title')}")
    print("내용 일부:", doc.page_content)
    print("코사인 유사도:", similarity)


질문 : 물체에 맞음 에 대한 안전조치사항 
정답 : 동일한 사고가 발생하지 않도록 안전교육 실시.

제목: 현수교 주탑시공 안전보건작업지침
내용 일부: 현수교 주탑 시공은 고소에서 작업이 이루어지는 경우가 많으므로 작업안전에 영향을 미칠 것으로 예상되는 다음사항에 대하여 안전작업계획서를 작성하여야 한다.

(가) 안전난간, 안전방망, 안전대 부착설비, 안전한 작업발판 등과 같은 떨어짐 방지대책
(나) 자재나 공구류의 맞음에 대한 재해를 예방할 수 있도록 출입금지구역 설정, 틈이 없는 바닥판 구조, 수직보호망, 방호선반 등과 같은 맞음 방지대책
(다) 크레인, 리프트 등을 사용하여 조립 및 해체작업을 하는 경우 작업방법 및 순서 등이 포함된 중량물 취급 계획

6. 현수교 주탑 작업 안전조치사항

6.1 공통사항

(1) 작업시작 전에 관리감독자를 지정하여 작업을 지휘하도록 하여야 한다.

(가) 주탑 작업에 투입되기 전 안전모, 안전대 등 근로자 개인보호구 점검 및 작업 중 착용여부 및 상태 확인
(나) 주탑 작업에 사용되는 가설장비는 사전에 안전점검을 실시하고 이상이 발견된 때에는 정상적인 장비로 교체하거나 정비하여 이상이 없음을 확인한 후 사용
(다) 위험기계 · 기구의 방호장치를 점검하고 이상이 있는 경우에는 정상적인 제품으로 교체
(라) 당해 작업의 위험요인을 점검하고 작업시작 전에 작업동료, 안전방망, 안전난간 등 안전시설의 설치상태와 이상유무 확인
(마) 작업장 내 공구 및 자재는 결속하여 정리정돈을 실시하여 맞음 등의 재해 예방
(바) 근로자에게 적절한 휴식시간 부여로 근골격계질환 예방을 위한 적절한 조치
(사) 주탑 작업에 대한 안전수칙을 근로자에게 주지시키고 이행여부 확인
코사인 유사도: 0.45916376099044576

제목: 안전대 사용지침
내용 일부: (다) 추락하여 매달려 혼들리는 경우 물체에 충돌하지 않도록 사용하여야 한다.

(라) 합성섬유로우프를 지지로우프로 사용하는 경우 추락 저지시 아래부분의 장애물에 접촉되지 않도록 사용하